In [1]:
import numpy as np


class NeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.01):
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.parameters = {}
        self.initialize_parameters()

    #初始化权重，seed作为种子控制权重每次生成一致，保证相同数据下训练得出的结论相同
    #每一层的权重都会受到上一层的神经元（layer_size）的影响，根据上一层神经元的大小控制本层权重的大小
    #其中计算权重的方法能够实现“均值 0、方差合适”的权重初始化（*后面的数据是缩放因子->方差调节器，每个激化函数都有对应的调节器）
    #若方差较大（权重计算方法不对），则对一层一层传递下去，导致输出的隐藏值过大，损失值爆炸，因此缩放因子不能随意更改
    #权重有正有负 —— 正权重代表 “增强输入信号”，负权重代表 “削弱输入信号
    def initialize_parameters(self):
        np.random.seed(42)
        for l in range(1, len(self.layer_sizes)):
            self.parameters[f"W{l}"] = np.random.randn(
                self.layer_sizes[l], self.layer_sizes[l - 1]) * np.sqrt(2.0 / self.layer_sizes[l - 1])
            self.parameters[f"b{l}"] = np.zeros(self.layer_sizes[l])

    #激化函数，该函数是完美适配的，能够将产生的负数据全部移除，避免后续产生梯度消失的情况
    #负数据的产生：权重 W 有负数，线性计算Z=W×A_prev + b中 “削弱作用超过增强作用” 的自然结果；
    def relu(self, Z):
        return np.maximum(0, Z)

    #梯度开关，关闭负数据的梯度，避免产生梯度消失问题
    def relu_derivative(self, Z):
        #反向传播的 “梯度开关”(0->梯度废弃，1->梯度下传)
        return (Z > 0).astype(float)

    #将获取的隐藏层数据Z转化成概率分布数据（所有隐藏层获取的数据为1，相当于对真实值每个位置的预测数据）
    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        #将我们预测的样本总数据变成总和为1的概率分布，用每个样本的数据除以样本总数据和
        return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

    #前向传播获取到预测的输出数据，并将隐藏层数据进行激化，输出层数据进行softmax，保存在缓存中用于后续反向传播
    #前向传播公式：Z=W*A(l-1)+b
    def forward_propagation(self, X):
        cache = {"A0": X}
        L = len(self.parameters) / 2
        for l in range(1, L):
            cache[f"Z{l}"] = np.dot(self.parameters[f"W{l}"], cache[f"A{l - 1}"]) + self.parameters[f"b{l}"]
            Z = self.relu(cache[f"Z{l}"])

        #Z是代表的被激活后的数据，A则是代表的原始权重、偏置后的数据以及输入和最后softmax处理后的输出层数据
        cache[f"Z{L}"] = np.dot(self.parameters[f"W{L}"], self.parameters[f"A{L - 1}"]) + self.parameters[f"b{L}"]
        cache[f"A{L - 1}"] = self.softmax(cache[f"Z{L}"])

        return cache


    def compute_loss(self, AL, Y):
        m = Y.shape[1]
        #因为Y是one-hot值，所以只有真实数据所在位置为1，其它位置都为零，这样就可以算出AL对应真实数据的概率，把所有概率相加（因为Log（AL）为负数，所有前面加-改为正数直观看出和真实数据的差距）
        loss = -np.sum(Y * np.log(AL + 1e-8)) / m
        return loss

    #输出层是 “概率输出 + 交叉熵损失” 的组合，梯度有专属简化公式；隐藏层是 “ReLU 激活”，梯度需要 “接收下一层梯度 + 筛选”，两者逻辑不同，必须分开处理。
    #反向传播：通过预测的输出数据一层一层向上推导，计算每一层权重的导数（导数越大说明对当前数据影响越偏离，对最终数据影响越大）
    def backward_propagation(self, X, Y, cache):
        m=X.shape[1]
        L=len(self.layer_sizes) // 2
        grads={}

        #Softmax + 交叉熵求导简化公式
        dZ=cache[f"A{L}"]-Y
        #把包装车间的误差，通过 “加工方式 W3” 倒推到第二道车间的输出 A2
        grads[f"dW{L}"]=np.dot(dZ,cache[f"A{L - 1}"].T)/m

        for l in range(1, L):
            dA=np.dot(self.parameters[f"W{l+1}"], dZ)
            dZ=dA*self.relu_derivative(cache[f"Z{l}"])
            grads[f"dW{l}"]=np.dot(dZ, cache[f"A{l-1}"].T)/m
            grads[f"db{l}"]=np.sum(dZ, axis=1, keepdims=True)/m

        return grads

    #根据反向传播计算出的每一层权重偏导数重新更新当前权重，通过学习率调节更新力度
    def update_paramaters(self,grads):
        L=len(self.layer_sizes) // 2
        for l in range(1, L):
            self.parameters[f"W{l}"]-= self.learning_rate * grads[f"dW{l}"]
            self.parameters[f"b{l}"]-= self.learning_rate * grads[f"db{l}"]


    def prdict(self,X):
        cache = self.forward_propagation(X)
        L=len(self.layer_sizes) // 2
        predictions=np.argmax(cache[f"A{L}"], axis=0)
        return predictions

    #argmax函数相当于将预测数据转化为one-hot样式，6000个样本中每个预测的数据位置，和样本中真实值的位置数据比较（true->1,false->0,再通过np.mean函数求取true和false所占的比例）
    def accuracy(self, X, Y):
        predictions=self.prdict(X)
        true_labels=np.argmax(Y, axis=0)
        accuracy=np.mean(predictions==true_labels)
        return accuracy

    #


